In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.148 --progress-bar off
!pip install -qqq openai==0.27.4 --progress-bar off
!pip install -qqq tiktoken==0.3.3 --progress-bar off
!pip install -qqq watermark==2.3.1 --progress-bar off
!pip install -qqq huggingface-hub==0.14.0 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off

In [51]:
import os
import textwrap
from getpass import getpass

import huggingface_hub
import langchain
import openai
from langchain import HuggingFaceHub, LLMChain, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [3]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 7.34.0

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.10.147+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

huggingface_hub: 0.14.0
langchain      : 0.0.148
openai         : 0.27.4



In [4]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [5]:
OPENAI_API_KEY = getpass()

··········


In [6]:
HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [7]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

## Large Language Models

In [8]:
template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

question = (
    "What is the relationship between Jim and Dwight from the TV show The Office?"
)

print(prompt.format(question=question))

Question: What is the relationship between Jim and Dwight from the TV show The Office?

Answer:


In [9]:
repo_id = "declare-lab/flan-alpaca-large"
flan = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0, "max_length": 64}
)

response = flan(prompt.format(question=question))
print_response(response)

Jim and Dwight are best friends and have a strong bond. They are often seen together at work, and
they often have a lot of fun together.


In [38]:
text_davinci_003 = OpenAI(temperature=0)
response = text_davinci_003(prompt.format(question=question))
print_response(response.strip())

Jim and Dwight have a complicated relationship. They are often at odds with each other, but they
also have a deep respect for one another. They are often seen as rivals, but they also have a strong
bond of friendship.


### Multiple Completions

In [40]:
text_davinci_003 = OpenAI(temperature=0.4, n=3, best_of=3)
response = text_davinci_003.generate([prompt.format(question=question)])
response

LLMResult(generations=[[Generation(text=' Jim and Dwight are colleagues and rivals. They often butt heads in the office, but they also have a strange sort of friendship. Dwight often looks to Jim for guidance and advice, and Jim occasionally shows a softer side to Dwight.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text=' Jim and Dwight are co-workers at the fictional paper company Dunder Mifflin in the TV show The Office. They have a strained relationship, as Jim often pranks Dwight and Dwight is often overly serious and strict. Despite their differences, they have a mutual respect for each other.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text=' Jim and Dwight have a complicated relationship. They are often at odds with each other, but there is also a strong underlying respect between them. They are often seen competing and pranking each other, but they also have moments of camaraderie.', generation_info={'finish_reason':

In [41]:
for i, generation in enumerate(response.generations[0]):
    print(f"Generation {i + 1}\n")
    print_response(generation.text.strip())
    print()

Generation 1

Jim and Dwight are colleagues and rivals. They often butt heads in the office, but they also have a
strange sort of friendship. Dwight often looks to Jim for guidance and advice, and Jim occasionally
shows a softer side to Dwight.

Generation 2

Jim and Dwight are co-workers at the fictional paper company Dunder Mifflin in the TV show The
Office. They have a strained relationship, as Jim often pranks Dwight and Dwight is often overly
serious and strict. Despite their differences, they have a mutual respect for each other.

Generation 3

Jim and Dwight have a complicated relationship. They are often at odds with each other, but there is
also a strong underlying respect between them. They are often seen competing and pranking each
other, but they also have moments of camaraderie.



## Embeddings

In [26]:
text = """
At the start of The Office, it was apparent that Dwight and Jim rarely got along. 
They were too competitive and too different of people to get along. 
But as the show goes on, time proves to be the one thing that holds Dwight and Jim together. 
They had been through too much as coworkers and friends to not give in to a friendship.

Aside from the toxic (albeit hilarious) relationship between Michael and Toby, 
Dwight and Jim come in second place with the most destructive relationship. 
From pranking each other to being a part of each other's weddings, 
Jim and Dwight have been through the wringer as friends.

The first season only touched the surface of Dwight and Jim’s friendship. 
Viewers saw how cutthroat they could be when Dwight stole Jim’s biggest sale of 
the year in ‘Diversity Day.’ Jim got revenge on Dwight, of course, in ‘Healthcare,’ 
when he made up various diseases that he and Pam had when Dwight asked for 
everyone’s medical history. By the end of the short season, Dwight shot his shot 
with Katy—the purse girl—but Jim was the one who ended up taking her home.

Looking at the ups and downs of Dwight and Jim’s friendship, it’s surprising 
that Jim invited Dwight to his house party in ‘Email Survalience.’ But things 
get back to normal when Jim gave Dwight bad advice when it was announced that 
Dwight had to speak at a salesman convention. The advice Jim gave Dwight was in 
jest but it ended up working for Dwight in the end.
"""

source: https://screenrant.com/jim-halpert-dwight-schrute-friendship-timeline-the-office/#season-1-cutthroat-friends

In [39]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [40]:
embedding = hf_embeddings.embed_query(text)

In [41]:
len(embedding)

384

In [42]:
embedding[:5]

[-0.13613401353359222,
 0.034860432147979736,
 0.05075468495488167,
 -0.06413132697343826,
 -0.05491665005683899]

In [43]:
openai_embeddings = OpenAIEmbeddings()

In [44]:
embedding = openai_embeddings.embed_query(text)

In [45]:
len(embedding)

1536

In [46]:
embedding[:5]

[-0.01814699370231159,
 -0.013020798642834074,
 0.024346111489793782,
 -0.033644788171017063,
 0.0029091483623929935]

## Chat Models

In [47]:
chat_gpt = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

question = (
    "What is the relationship between Jim and Dwight from the TV show The Office?"
)

response = chat_gpt([HumanMessage(content=question)])
response

AIMessage(content='As an AI language model, I do not have personal opinions or beliefs, but I can provide information. Jim and Dwight have a complex relationship throughout the TV show The Office. At the beginning of the series, they are portrayed as bitter rivals, with Jim often playing pranks on Dwight. However, as the show progresses, they develop a mutual respect for each other and become friends. They still play pranks on each other, but it is done in a more lighthearted and playful manner. Overall, their relationship is a mix of competition, humor, and friendship.', additional_kwargs={})

In [48]:
print_response(response.content)

As an AI language model, I do not have personal opinions or beliefs, but I can provide information.
Jim and Dwight have a complex relationship throughout the TV show The Office. At the beginning of
the series, they are portrayed as bitter rivals, with Jim often playing pranks on Dwight. However,
as the show progresses, they develop a mutual respect for each other and become friends. They still
play pranks on each other, but it is done in a more lighthearted and playful manner. Overall, their
relationship is a mix of competition, humor, and friendship.


In [50]:
messages = [
    SystemMessage(content="You're an expert on the TV show The Office."),
    HumanMessage(content="What is the relationship between Jim and Dwight?"),
]
response = chat_gpt(messages)
print_response(response.content)

Jim and Dwight have a complicated relationship throughout the show. At the beginning, they are
rivals and constantly play pranks on each other. However, as the show progresses, they develop a
mutual respect for each other and become friends. Jim even becomes Dwight's best man at his wedding.
Despite their friendship, they still enjoy playing pranks on each other from time to time.


In [53]:
question = "What is the relationship between Jim and Dwight?"

In [57]:
system_template = "You're an expert on the TV show The Office. You {style}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)
chat_prompt.format_prompt(
    style="reply in thoughtful and philosophical manner", question=question
).to_messages()

[SystemMessage(content="You're an expert on the TV show The Office. You reply in thoughtful and philosophical manner.", additional_kwargs={}),
 HumanMessage(content='What is the relationship between Jim and Dwight?', additional_kwargs={})]

In [58]:
messages = chat_prompt.format_prompt(
    style="reply in thoughtful and philosophical manner", question=question
).to_messages()
response = chat_gpt(messages)
print_response(response.content)

The relationship between Jim and Dwight on The Office is a complex one that evolves over the course
of the series. At the beginning, they are portrayed as antagonistic towards each other, with Jim
often playing pranks on Dwight and Dwight taking his job as Assistant to the Regional Manager very
seriously. However, as the series progresses, we see moments of genuine friendship and understanding
between the two characters.  One could argue that Jim and Dwight represent two different approaches
to life and work. Jim is more laid-back and focused on enjoying his job and relationships with his
coworkers, while Dwight is more intense and driven by a desire to succeed and be recognized for his
hard work. Despite their differences, they both share a deep commitment to their jobs and a desire
to be respected by their colleagues.  Ultimately, the relationship between Jim and Dwight is a
testament to the power of human connection and the ability to find common ground with those who may
seem very 

In [59]:
messages = chat_prompt.format_prompt(
    style="reply in sarcastic and outrageous manner", question=question
).to_messages()
response = chat_gpt(messages)
print_response(response.content)

Oh, you mean the dynamic duo of the Scranton branch? Well, Jim is obviously Dwight's mortal enemy
and arch-nemesis. They spend their days plotting against each other and engaging in epic battles of
wits and pranks. It's like a modern-day version of Batman and the Joker, but with more staplers in
Jello.
